# ezlocalai with ngrok

Open this notebook in [Google Colab.](https://colab.research.google.com/)

Set the resources to T4 GPU, free tier.

[Get your free NGROK_TOKEN here.](https://dashboard.ngrok.com/get-started/your-authtoken) and add it to the colab secrets (Key logo on the left in Google Colab.)

It takes several minutes to install dependencies and to start the server. You can see the ngrok URL in the cell output.


In [ ]:
!apt-get update --fix-missing && apt-get upgrade -y
!apt-get install -y --fix-missing --no-install-recommends git build-essential cmake gcc g++ portaudio19-dev ffmpeg libportaudio2 libasound-dev python3 python3-pip wget ocl-icd-opencl-dev opencl-headers clinfo libclblast-dev libopenblas-dev ninja-build python3.10-dev
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!ln -s /usr/bin/python3 /usr/bin/python
!apt-get clean && rm -rf /var/lib/apt/lists/* /var/cache/apt/* /tmp/* /var/tmp/*
!python3 -m pip install --upgrade pip cmake scikit-build setuptools wheel pyngrok --no-cache-dir
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir
!rm -rf sample_data .config
!git clone https://github.com/DevXT-LLC/ezlocalai .
!pip install -r cuda-requirements.txt
!sed -i 's/WHISPER_MODEL=base.en/WHISPER_MODEL=large-v3/g' .env

In [ ]:
# Add your NGROK_TOKEN to your colab secrets if using Google Colab (Key logo on the left)
try:
  from google.colab import userdata
  NGROK_TOKEN = userdata.get('NGROK_TOKEN')
  if not NGROK_TOKEN:
    raise
except:
  # If you're not using Google Colab, enter your NGROK_TOKEN below.
  NGROK_TOKEN = "YOUR NGROK_TOKEN HERE"
from pyngrok import ngrok
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(8091)
print(f"Public URL: {public_url}")
print(f"Please wait for the server to say it is ready before trying to connect externally.")
!uvicorn app:app --host 0.0.0.0 --port 8091 --workers 1 --proxy-headers
